# Sumatra export

In [224]:
import json
import pandas
import numpy as np

!smt export
with open('.smt/records_export.json') as ff:
    data = json.load(ff)
    
with open('record1.json', 'w') as record1:
    for entry in range(len(data)):
        record1.write(json.dumps(data[entry], sort_keys=True, indent=4, separators=(',', ': ')))

df = pandas.DataFrame(data)  #df is now the sumatra dataframe

# Extractor() : 

This method finds the simulations which have a single ".mpz.npz" file in their Data/ folder and gets all the information out of that file into a dataframe

In [262]:
import glob
import os
import fipy as fp

# print df['output_data'][0][-2]['path']
# df['datastore'][0]['parameters']['root']  #alternative method to get data directory

labels = []

labels = df['label']
print labels

dfC = pandas.DataFrame(columns=['label','steps','elapsed', 'nx', 'ny', 'c_var_array', 'cvar_Interpolated_Function', 'dt', 'dx', 'dy', 'sweeps'])

def extractor(labelz, Lx=200):  #will work as long as there is 1 .mpz.npz file in the simulation directory
    rows = 0
    for lbl in labelz:
        dictt = {} #create a dictionary to fill in with simulation data
        dictt.update({'label':lbl})
        datapath = os.path.join('/data/aem1/new1a/surf-research/Andrey/Data', lbl)
        mfile = glob.glob('{0}/*.mpz.npz'.format(datapath)) #get the name of the mpz file with data we need
        
        if len(mfile)>=1:
            fn = np.load(mfile[0])
            for item in fn:
                dictt.update({str(item):fn[item]}) #add every variable from the file into the dictionary
            dictt.update({'cvar_Interpolated_Function':None}) #create a blank column for later
            
            #now we need the x and y coordinates for interpolation to match the grid size
            nx = dictt['nx']
            ny = dictt['ny']

            print dictt.keys()
            dfC.loc[rows]=dictt        
        rows+=1
    return dfC #dfC is a compact dataframe with all the information we need
        
extractor(labels)

0    7e0c29789354
1    48f844a8f9ca
2    3b51d3753183
Name: label, dtype: object
['nx', 'ny', 'c_var_array', 'dx', 'dy', 'steps', 'dt', 'label', 'cvar_Interpolated_Function', 'elapsed']
['nx', 'ny', 'c_var_array', 'dx', 'dy', 'steps', 'dt', 'label', 'cvar_Interpolated_Function', 'elapsed']


,label,steps,elapsed,nx,ny,c_var_array,cvar_Interpolated_Function,dt,dx,dy
1,48f844a8f9ca,19,0.5,200,200,"[0.50946391768, 0.512896902025, 0.515078121617...",None,0.0440082686508,1.0,1
2,3b51d3753183,19,0.5,100,100,"[0.517433869441, 0.522449680227, 0.51992540421...",None,0.0440082686508,2.0,2


# cvar Interpolation

The method below will take the datafram which is created by the extractor() above and save the interpolation function into the dataframe

In [269]:
print 'This import takes a while and I dont know why'
from scipy import interpolate as scpinter
#from scipy.interpolate import interp2d as scpinter    
print 'importing done'
def cvarInterp(cvarFrame, Lx = 200):
    for variables in dfC.iterrows():
        dictt = {}
        print variables
        nx = variables[1]['nx']
        ny = variables[1]['ny']
        m = fp.Grid2D(nx = nx, ny= ny, dx=Lx / nx, dy=Lx / ny)
        x = m.x
        y = m.y
#         dictt.update({'x':m.x})
#         dictt.update({'y':m.y})
        cvar = variables[1]['c_var_array']
        function = scpinter(x, y, cvar, copy=False)
        variables[1]['cvar_Interpolated_Function'] = function #set the interpolated function into the dataframe column
cvarInterp(dfC)

importing done
(1, label                                                              48f844a8f9ca
steps                                                                        19
elapsed                                                                     0.5
nx                                                                          200
ny                                                                          200
c_var_array                   [0.50946391768, 0.512896902025, 0.515078121617...
cvar_Interpolated_Function    <scipy.interpolate.interpolate.interp2d object...
dt                                                              0.0440082686508
dx                                                                          1.0
dy                                                                            1
Name: 1, dtype: object)
whats taking so long
almost done
done
(2, label                                                              3b51d3753183
steps                              

In [270]:
dfC

,label,steps,elapsed,nx,ny,c_var_array,cvar_Interpolated_Function,dt,dx,dy
1,48f844a8f9ca,19,0.5,200,200,"[0.50946391768, 0.512896902025, 0.515078121617...",<scipy.interpolate.interpolate.interp2d object...,0.0440082686508,1.0,1
2,3b51d3753183,19,0.5,100,100,"[0.517433869441, 0.522449680227, 0.51992540421...",<scipy.interpolate.interpolate.interp2d object...,0.0440082686508,2.0,2


In [245]:
for x in dfC.iterrows():
    print x

(1, label                                               48f844a8f9ca
steps                                                         19
elapsed                                                      0.5
nx                                                           200
ny                                                           200
c_var_array    [0.50946391768, 0.512896902025, 0.515078121617...
cvarInterp     <scipy.interpolate.interpolate.interp2d object...
dt                                               0.0440082686508
dx                                                           1.0
dy                                                             1
Name: 1, dtype: object)
(2, label                                               3b51d3753183
steps                                                         19
elapsed                                                      0.5
nx                                                           100
ny                                                        

# cvar Figures of Merit

Now we have everything we need in the dataframe and can compare the different grid sizes, calculate L2 using cvar etc

In [ ]:
cvar_times = []
#smtfram is the datafram created by sumatra records
#dframe is the table we create from .mpz.npz files because it can have multiple entries for the same simulation in order to compare the concentration values at different times
def CvarCompare(dframe, smtframe, Lx=200):
    L1_cvar_Norms = [] #should these be dictionaries to match with the proper simulation label? 
    L2_cvar_Norms = []
    Linf_cvar_Norms = []
    N_Best = 0
    sweeps_Best = 0
    
    labels = smtframe['label']
    times = list(set(int(dframe['elapsed']))  #a list of the unique times where we sampled c_var
    L1_cvar_times = dict(('L1_cvar_time_{0}'.format(time), []) for time in times)
    L2_cvar_times = dict(('L2_cvar_time_{0}'.format(time), []) for time in times)
    Linf_cvar_times = dict(('Linf_cvar_time_{0}'.format(time), []) for time in times)

    #first we need to figure out which is the finest grid to set it as the standard of comparison
    for row in dframe.iterrows():
        if row[1]['nx']>N_Best and row[1]['sweeps']>sweeps_Best:
            Best_cvar_function = row[1]['cvar_Interpolated_Function']
            N_Best = row[1]['nx']
            sweeps_Best = row[1]['sweeps']
    #now for every sumatra label, go thorugh the dframe, and make lists for each norm and each sampled time 
    for label in labels:
        for row in dframe.iterrows():  
            if row[1]['label']==label:
                cvar_function = row[1]['cvar_Interpolated_Function']
                elapsed = row[1]['elapsed']
                nx = row[1]['nx']
                ny = row[1]['ny']
                m = fp.Grid2D(nx = nx, ny= ny, dx=Lx / nx, dy=Lx / ny)
                x = m.x
                y = m.y
                diff = np.absolute(cvar_function[x,y] - Best_cvar_function[x,y])
                
                L1_cvar_times['L1_cvar_time_{0}'.format(elapsed)].append(np.linalg.norm(diff,1))
                L2_cvar_times['L2_cvar_time_{0}'.format(elapsed)].append(np.linalg.norm(diff,2))
                Linf_cvar_times['Linf_cvar_time_{0}'.format(elapsed)].append(np.linalg.norm(diff,np.inf))

                
    #add the norms data to the sumatra database as seperate columns             
    for key in L1_cvar_times.keys():
        smtframe[key] = L1_cvar_times[key]
    for key in L2_cvar_times.keys():
        smtframe[key] = L2_cvar_times[key]         
    for key in Linf_cvar_times.keys():
        smtframe[key] = Linf_cvar_times[key]         
#                 try:
#                     L1_cvar_time_{0}.format(elapsed).append(np.linalg.norm(diff,1))
#                     L2_cvar_time_{0}.format(elapsed).append(np.linalg.norm(diff,2))
#                     L3_cvar_time_{0}.format(elapsed).append(np.linalg.norm(diff,3))
#                 except:
#                     L1_cvar_time_{0}.format(elapsed)=[np.linalg.norm(diff,1)]
#                     L2_cvar_time_{0}.format(elapsed)=[np.linalg.norm(diff,2)]
#                     L3_cvar_time_{0}.format(elapsed)=[np.linalg.norm(diff,3)]
#     for x in times:
#         smtframe['L1_cvar_time_{0}'.format(x)] = L1_cvar_time_{0}.format(x)
#         smtframe['L2_cvar_time_{0}'.format(x)] = L2_cvar_time_{0}.format(x)
#         smtframe['L3_cvar_time_{0}'.format(x)] = L3_cvar_time_{0}.format(x)
 
        
#     smtframe['L1 cvar Norms time:{0}'] = L1Norms
#     smtframe['L2 Norms'] = L2Norms
#     smtframe['L-infinite Norms'] = LinfNorms
#     return True

# Free Energy Interpolation

For this section, we have to take the elapsed time and cvar from each step file, calculate the energy at that step, then interpolate the array of energies along with time. Save the intepolated function into the SUMATRA dataframe

In [223]:
energyFunctions = []
energyFunctions.append(None)
print energyFunctions
print ' '
for row in df.iterrows():
    print row[1]['label']
    
df

#How to select a specific row and then you can change the values in it's columns
df.loc[df['label']=='bf22b1bfaef6']['datastore']

[None]
 
48841eea4a91
da25c86b72f0
bf22b1bfaef6
7f52183544f2
47235abe3a7b


2    {u'type': u'FileSystemDataStore', u'parameters...
Name: datastore, dtype: object

In [272]:
import os
import numpy as np
from scipy import interpolate as scpinter

results = {}
c_alpha = 0.3
c_beta = 0.7
kappa = 2.0
# M = 5.0
# c_0 = 0.5
# epsilon = 0.01
rho_s = 5.0

def f_0(c):
    return rho_s*((c - c_alpha)**2)*((c_beta-c)**2)

def f(c):
    return (f_0(c)+ .5*kappa*(np.linalg.norm(np.gradient(c)))**2)


def freeEnergyInterp(labels, dframe, Lx = 200):
    energyFunctions = []
    #This code will access every step file saved in the Data/[label] directory 
    for label in dframe['label']:
        e_elaps = []
        e_enrg = []
        filepath = os.path.join('/data/aem1/new1a/surf-research/Andrey/Data', label)
        #gets the last step file in the directory (hopefully at equillibrium)
        file = os.listdir('Data/{0}'.format(label))[len(os.listdir('Data/{0}'.format(label)))-2] 
        stepfiles = glob.glob('{0}/*.npz'.format(filepath)) #get the list of all step files for simulation
        print stepfiles
        if len(stepfiles)==0: 
            energyFunctions.append(None)
        else:
            for stpfile in stepfiles:
                fn = np.load(stpfile)
                #do free enrgy math here

                cvar_array = fn['c_var_array']
                cells = fn['nx']*fn['ny']
                dx = Lx / fn['nx']
                dy = Lx / fn['ny']
                free_energy_array = f(cvar_array)
                free_energy_cellVolAvg = np.mean(free_energy_array)*cells*dx*dy
#                 f(c_var).cellVolumeAverage*cells*(dx**2)


                e_elaps.append(fn['elapsed'])
                e_enrg.append(free_energy_cellVolAvg)

            eFunction = scpinter.interp1d(e_elaps, e_enrg, copy=False)
            energyFunctions.append(eFunction) #save a list of all the energy functions
        print 'Number of energy functions: ', len(energyFunctions)    
    dframe['Free_Energy_Interpolated_Function'] = energyFunctions #add the list as a new column to the dataframe

labels = dfC['label']
print labels
freeEnergyInterp(labels, dfC)
#             data = {'cvar':fn['cvar'], 'freeEn':float(fn['f']), 'step':int(fn['steps']), 'elapsed':float(fn['time'])}
#     #         print type(data)
#     #         print type(str(fn['grid']))
#             results.update({str(fn['grid']):data})


1    48f844a8f9ca
2    3b51d3753183
Name: label, dtype: object
['/data/aem1/new1a/surf-research/Andrey/Data/48f844a8f9ca/1a_200_step000000_data_time-200.00.npz', '/data/aem1/new1a/surf-research/Andrey/Data/48f844a8f9ca/data_time-0.50_step-000018.mpz.npz', '/data/aem1/new1a/surf-research/Andrey/Data/48f844a8f9ca/data_time-0.56_step-000019.npz', '/data/aem1/new1a/surf-research/Andrey/Data/48f844a8f9ca/data_time-1.00_step-000025.mpz.npz', '/data/aem1/new1a/surf-research/Andrey/Data/48f844a8f9ca/data_time-1.10_step-000026.npz', '/data/aem1/new1a/surf-research/Andrey/Data/48f844a8f9ca/data_time-10.00_step-000051.mpz.npz', '/data/aem1/new1a/surf-research/Andrey/Data/48f844a8f9ca/data_time-10.68_step-000052.npz']
Number of energy functions:  1
['/data/aem1/new1a/surf-research/Andrey/Data/3b51d3753183/1a_100_step000000_data_time-100.00.npz', '/data/aem1/new1a/surf-research/Andrey/Data/3b51d3753183/data_time-0.50_step-000018.mpz.npz', '/data/aem1/new1a/surf-research/Andrey/Data/3b51d3753183/dat

In [273]:
dfC

,label,steps,elapsed,nx,ny,c_var_array,cvar_Interpolated_Function,dt,dx,dy,Free_Energy_Interpolated_Function
1,48f844a8f9ca,19,0.5,200,200,"[0.50946391768, 0.512896902025, 0.515078121617...",<scipy.interpolate.interpolate.interp2d object...,0.0440082686508,1.0,1,<scipy.interpolate.interpolate.interp1d object...
2,3b51d3753183,19,0.5,100,100,"[0.517433869441, 0.522449680227, 0.51992540421...",<scipy.interpolate.interpolate.interp2d object...,0.0440082686508,2.0,2,<scipy.interpolate.interpolate.interp1d object...


# Figures of Merit

Now we need to use methods to add some figures of merit to the dataframes to compare the grids to the 400 grid

In [ ]:
for row in df.iterrows():

In [219]:
#add moose data as well
#This method takes any set of "ideal times" and compares the free energy values at those times for every simulation

ideal_times = []
for i in range(160): ideal_times.append(i)
    
def FreeEnergyCompare(dframe, ideal_times, params.json, smtframe):
    L1Norms = [] #should these be dictionaries to match with the proper simulation label? 
    L2Norms = []
    LinfNorms = []
    N_Best = 0
    sweeps_Best = 0
    
    for row in dframe.iterrows():
        if row[1]['nx']>N_Best and row[1]['sweeps']>sweeps_Best:
            BestEnergyFunct = row[1]['Free_Energy_Function']
            N_Best = row[1]['nx']
            sweeps_Best = row[1]['sweeps']
    for row in dframe.iterrows():    
        efunction = row[1]['Free_Energy_Function']
        diff = np.absolute(efunction[ideal_times] - BestEnergyFunct[ideal_times])
        L1Norms.append(np.linalg.norm(diff,1)) 
        L2Norms.append(np.linalg.norm(diff,2)) 
        LinfNorms.append(np.linalg.norm(diff, np.inf)) 
    
    
    smtframe['L1 Free Energy Norms'] = L1Norms
    smtframe['L2 Free Energy Norms'] = L2Norms
    smtframe['L-infinite Free Energy Norms'] = LinfNorms
    #Anushkas code
#     diff_50.append(np.absolute(y50x4[ideal_times] - y400x5[ideal_times]))
#     L1Norms.append(np.linalg.norm(diff_50,1))
#     L2Norms.append(np.linalg.norm(diff_50,2))
#     LinfNorms.append(np.linalg.norm(diff_50, np.inf))
    

IndentationError: expected an indented block (<ipython-input-219-2a4d61f10dc9>, line 4)

In [36]:
results['10x1']['freeEn']

0.7865168606271677

In [30]:
print len(df['output_data'][0])

4


In [31]:
for p in 'N', 'steps', 'sweeps':
    df[p] = df.parameters.map(lambda x: json.loads(x['content'])[p])

In [ ]:
print df['parameters'][0]['content']

In [37]:
df['Free Energy']=results['10x1']['freeEn']
df['elapsed']=results['10x1']['elapsed']

columns.append('Free Energy')
columns.append('elapsed')
df[columns]

,label,N,steps,tags,duration,sweeps,memory,Free Energy,elapsed
0,8f1f49c60b96,10,20,[npzTest],4.042712,2,4648.0,0.786517,0.57275


In [33]:
columns = ['label', 'N', 'steps', 'tags', 'duration', 'sweeps']
print df[columns]

          label   N  steps       tags  duration  sweeps
0  8f1f49c60b96  10     20  [npzTest]  4.042712       2


In [34]:
import numpy as np
import os
num = 0
memoryArray = np.array([])
for num in range(len(df.index)):
    memName = df['output_data'][num][len(df['output_data'][num])-1]['path']
    memlocation = df['datastore'][num]['parameters']['root']
    pool = os.path.join(memlocation, memName)
    memdata = np.loadtxt(pool)
    memoryArray = np.append(memoryArray, memdata)
    print memoryArray
    print ' '
    print num

[ 4648.]
 
0


In [35]:
df['memory']=memoryArray
columns.append('memory')
df[columns]

,label,N,steps,tags,duration,sweeps,memory
0,8f1f49c60b96,10,20,[npzTest],4.042712,2,4648.0


In [38]:
data = {'a' : [1, 2, 3], 'b' : [4, 5, 6]}
pandas.DataFrame(data)

,a,b
0,1,4
1,2,5
2,3,6


In [59]:
data = [{'a' : 1, 'b' : 4}, {'a' : 2, 'b' : 5}, {'a' : 3 ,'b' : 6}]
aa  = pandas.DataFrame(data)

In [60]:
def recalc(row):
    return row.b * 2

In [61]:
print aa

   a  b
0  1  4
1  2  5
2  3  6


In [ ]:
aa['c'] = [recalc(row) for index, row in aa.iterrows()]
print aa

In [73]:
times = np.array([0., 1.0, 1.5])
values = times**2


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(times, values)

In [ ]:
import numpy as np
np.linspace(0., 100., 1000)

In [83]:
!ls Data/1de38ff12ec7/
import glob
glob.glob('Data/1de38ff12ec7/*.npz')

1a10x1step0.npz  1a10x1step10.npz  1a10x1step20.npz  memory_usage.txt


['Data/1de38ff12ec7/1a10x1step0.npz',
 'Data/1de38ff12ec7/1a10x1step10.npz',
 'Data/1de38ff12ec7/1a10x1step20.npz']